Pakete laden

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape
import requests

Funktionen

In [25]:
def download_tiff(url, save_path):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print("TIFF heruntergeladen und gespeichert unter:", save_path)
        else:
            print("Fehler beim Herunterladen der Datei. Statuscode:", response.status_code)
    except Exception as e:
        print("Fehler:", e)

In [ ]:
# Funktion, um Polygone aus einer Geometry Collection zu extrahieren
def extract_polygons(geometry):
    if geometry["type"] == "GeometryCollection":
        # Extrahiere Polygone und MultiPolygone aus der Geometry Collection
        return [shape(geom) for geom in geometry["geometries"] if geom["type"] in ["Polygon", "MultiPolygon"]]
    else:
        # Geometrie direkt zurückgeben, wenn sie kein GeometryCollection ist
        return [shape(geometry)]

# Daten beschaffen

Tiff

In [5]:
url = "https://bfe-ogd.s3.eu-central-1.amazonaws.com/2023-11-11%2B000_alps_SWE_product.tif"
save_path = "bild.tiff"
download_tiff(url, save_path)

TIFF heruntergeladen und gespeichert unter: bild.tiff


In [25]:
from osgeo import gdal, osr

# Pfad zu Ihrer GeoTIFF-Datei
file_path = r"C:\Hackathon\1_Rohdaten\bild.tiff"
dataset = gdal.Open(file_path)

# Koordinatensystem ermitteln
srs = osr.SpatialReference()
srs.ImportFromWkt(dataset.GetProjection())

# Koordinatensystem ausgeben
print("Koordinatensystem:")
print(srs.ExportToPrettyWkt())

Koordinatensystem:
PROJCS["WGS 84 / Pseudo-Mercator",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Mercator_1SP"],
    PARAMETER["central_meridian",0],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],
    AUTHORITY["EPSG","3857"]]


Standorte laden

In [2]:
locations = pd.read_csv(r"C:\Hackathon\1_Rohdaten\standorte.csv")

Standorte zu Shape transformieren und speichern

In [3]:
locations['geometry'] =  locations['geometry'].apply(wkt.loads)
point_gdf = gpd.GeoDataFrame(data=locations, geometry="geometry")
#point_gdf.to_file(r"C:\Hackathon\2_Processing\standorte.shp")

In [ ]:
# Koordinaten extrahieren
point_gdf['X'] = point_gdf['geometry'].x
point_gdf['Y'] = point_gdf['geometry'].y

In [ ]:
# Geometrie-Daten löschen
point_gdf = point_gdf.drop(columns="geometry")

Einzugsgebiete laden

In [21]:
# Pro Standort die Einzugsgebiete von Swisstopo-API beziehen
count = 1
print("Bearbeite Nr. ", count)
dataframelist = []

for x, y, xtf_id in zip(point_gdf["X"], point_gdf["Y"], point_gdf["xtf_id"]):
        data = requests.get(f"https://api3.geo.admin.ch/rest/services/all/MapServer/identify?geometry={x},{y}&geometryFormat=geojson&geometryType=esriGeometryPoint&imageDisplay=10,10,96&lang=de&layers=all:ch.bafu.wasser-teileinzugsgebiete_2&limit=10&mapExtent=2664777,1096799,2664787,1096809&returnGeometry=true&sr=2056&tolerance=1")
        data = data.json() # transform request in json format
        count = count + 1
        print("Bearbeite Nr. ", count)
        
        
        # Extrahieren der Polygon-Geometrien
        features = []
        for feature in data["results"]:
            geometries = extract_polygons(feature["geometry"])
            for geom in geometries:
                new_feature = feature.copy()
                new_feature["geometry"] = geom
                features.append(new_feature)

        # Konvertieren Sie die extrahierten Features in ein GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(features)
        gdf["xtf_id"] = xtf_id
        dataframelist.append(gdf)
                
rdf = gpd.GeoDataFrame(pd.concat(dataframelist, ignore_index=True))
rdf = rdf.merge(point_gdf, on="xtf_id", how="left")
gdf = gpd.GeoDataFrame(rdf)

Bearbeite Nr.  1
Bearbeite Nr.  2
Bearbeite Nr.  3
Bearbeite Nr.  4
Bearbeite Nr.  5
Bearbeite Nr.  6
Bearbeite Nr.  7
Bearbeite Nr.  8
Bearbeite Nr.  9
Bearbeite Nr.  10
Bearbeite Nr.  11
Bearbeite Nr.  12
Bearbeite Nr.  13
Bearbeite Nr.  14
Bearbeite Nr.  15
Bearbeite Nr.  16
Bearbeite Nr.  17
Bearbeite Nr.  18
Bearbeite Nr.  19
Bearbeite Nr.  20
Bearbeite Nr.  21
Bearbeite Nr.  22
Bearbeite Nr.  23
Bearbeite Nr.  24
Bearbeite Nr.  25
Bearbeite Nr.  26
Bearbeite Nr.  27
Bearbeite Nr.  28
Bearbeite Nr.  29
Bearbeite Nr.  30
Bearbeite Nr.  31
Bearbeite Nr.  32
Bearbeite Nr.  33
Bearbeite Nr.  34
Bearbeite Nr.  35
Bearbeite Nr.  36
Bearbeite Nr.  37
Bearbeite Nr.  38
Bearbeite Nr.  39
Bearbeite Nr.  40
Bearbeite Nr.  41
Bearbeite Nr.  42
Bearbeite Nr.  43
Bearbeite Nr.  44
Bearbeite Nr.  45
Bearbeite Nr.  46
Bearbeite Nr.  47
Bearbeite Nr.  48
Bearbeite Nr.  49
Bearbeite Nr.  50
Bearbeite Nr.  51
Bearbeite Nr.  52
Bearbeite Nr.  53
Bearbeite Nr.  54
Bearbeite Nr.  55
Bearbeite Nr.  56
B

C:\Users\fth\AppData\Local\Temp\ipykernel_7264\2793054125.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rdf = gpd.GeoDataFrame(pd.concat(dataframelist, ignore_index=True))


In [22]:
# Shapefile speichern
output_filename = r'C:\Hackathon\2_Processing\Einzugsgebiete\Einzugsgebiete_NEW.shp'
gdf.to_file(output_filename, driver='ESRI Shapefile')

C:\Users\fth\AppData\Local\Temp\ipykernel_7264\1595185985.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_filename, driver='ESRI Shapefile')


In [24]:
# Geopackage speichern
output_filename = r'C:\Hackathon\2_Processing\Einzugsgebiete\Einzugsgebiete_NEW.geojson'
rdf.to_file(output_filename, driver='GeoJSON')